# Code

## Different encoding techniques

### Bag of words
* Transforms words into numbers using a dictionay.
* We only care about the frequency of words in a sentence.
* Bad because it doesn't take into account the ordering of those words!

In [1]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


### Word embeddings
* Tranforms words into a N dimensional vector.
* Tries to make similar words point to similar directions and antonym words in opposite directions.
* It's a layer of the model.

## Setiment Analysis with Recurrent Neural Networks

In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 400
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
17473536/17464789 [==============================] - 0s 0us/step


In [3]:
train_data.shape

(25000,)

In [4]:
len(train_data[3])

550

Padding reviews so that they can be the same size.

In [5]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [6]:
len(train_data[3])

400

### Creating the model

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 16),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          1417344   
                                                                 
 lstm (LSTM)                 (None, 16)                2112      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,419,473
Trainable params: 1,419,473
Non-trainable params: 0
_________________________________________________________________


### Training

In [9]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 57s 81ms/step - loss: 0.4683 - acc: 0.7787 - val_loss: 0.3286 - val_acc: 0.8668
Epoch 2/10
625/625 [==============================] - 51s 82ms/step - loss: 0.2501 - acc: 0.9032 - val_loss: 0.2867 - val_acc: 0.8836
Epoch 3/10
625/625 [==============================] - 49s 79ms/step - loss: 0.1848 - acc: 0.9318 - val_loss: 0.2722 - val_acc: 0.8934
Epoch 4/10
625/625 [==============================] - 49s 79ms/step - loss: 0.1531 - acc: 0.9466 - val_loss: 0.3027 - val_acc: 0.8964
Epoch 5/10
625/625 [==============================] - 49s 79ms/step - loss: 0.1272 - acc: 0.9553 - val_loss: 0.3246 - val_acc: 0.8946
Epoch 6/10
625/625 [==============================] - 51s 82ms/step - loss: 0.1102 - acc: 0.9624 - val_loss: 0.2768 - val_acc: 0.8928
Epoch 7/10
625/625 [==============================] - 49s 79ms/step - loss: 0.1001 - acc: 0.9653 - val_loss: 0.4219 - val_acc: 0.8824
Epoch 8/10
625/625 [==============================] - 50s 79ms

KeyboardInterrupt: ignored

In [10]:
model.evaluate(test_data, test_labels)

782/782 [==============================] - 18s 23ms/step - loss: 0.4695 - acc: 0.8480


[0.4694586992263794, 0.848039984703064]

### Predicting

In [11]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [12]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [15]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,MAXLEN))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was fire, really loved it and would watch it again because it was great"
predict(positive_review)

negative_review = "That movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

review = "really nice movie nice nice nice nice nice nice nice nice nice nice"
predict(review)

[0.9423091]
[0.44977587]
[0.780455]


## Play Generator with RNN

In [16]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


### Dataset

In [17]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


### Reading file content

In [18]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
'Length of text: {} characters'.format(len(text))

'Length of text: 1115394 characters'

In [22]:
print(text[:280])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

F


### Encoding text

In [23]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [25]:
# lets look at how part of our text is encoded
max_index = 13
print("Text:", text[:max_index])
print("Encoded:", text_to_int(text[:max_index]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [27]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:max_index]))

First Citizen


### Creating training examples

In [28]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [29]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [30]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [31]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [32]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Building the model

In [33]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Creating the loss function

In [34]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

In [36]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00123202  0.00097746  0.00101778 ...  0.00276049  0.00184684
  -0.00332387]
 [-0.00216997  0.0024099   0.00220178 ...  0.00435392  0.00351144
  -0.00552667]
 [-0.00233137  0.00326997  0.00224819 ... -0.00184656  0.00074331
  -0.00148076]
 ...
 [-0.00563261  0.0114264   0.01418458 ... -0.00678292 -0.0086744
  -0.00274013]
 [-0.00923069  0.0065335   0.01315108 ... -0.00440346 -0.00626947
   0.00104885]
 [-0.00992797  0.00795599  0.00559041 ... -0.00838242 -0.00490273
   0.000685  ]], shape=(100, 65), dtype=float32)


In [37]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-0.00123202  0.00097746  0.00101778  0.00104992  0.00510853  0.00259233
  0.00207283 -0.00449835  0.0026038   0.00436174  0.0001319  -0.00139873
 -0.00450894 -0.00300498 -0.00336798 -0.0045438  -0.00343565 -0.00141875
  0.00138818  0.0010282   0.00538143  0.00257647 -0.00293807  0.000276
  0.00023713 -0.00764048 -0.00262197 -0.00073794 -0.00056239  0.00157786
 -0.00347422  0.00151969 -0.00332236  0.00057631 -0.00231169 -0.00410942
  0.00045926 -0.00443494 -0.00159698  0.00152113 -0.00078343  0.00523129
  0.00467792 -0.00203488  0.00111008 -0.00595684  0.00188871 -0.00039446
 -0.002462   -0.00461904 -0.00212037 -0.00335748 -0.00185713 -0.00033557
  0.00248588  0.00243644  0.0037819  -0.00026572  0.00125178 -0.00012109
  0.00228718  0.00510106  0.00276049  0.00184684 -0.00332387], shape=(65,), dtype=float32)


In [38]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"'tS?ceA$uJZPesvoEMdWAZ'EVFXvH,qzj;kZ\nHonF!AdGMLXydOIQc, zX-j\nqK \nJM?SwF!\nUCQ'i;-Pr-t:a;qQBnEKHgKHNRi"

In [39]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Compiling the model

In [42]:
model.compile(optimizer='adam', loss=loss)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Setting up checkpoints

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Training

In [44]:
EPOCHS = 20

In [45]:
history = model.fit(data, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 31s 159ms/step - loss: 2.6054
Epoch 2/20
172/172 [==============================] - 30s 161ms/step - loss: 1.8979
Epoch 3/20
172/172 [==============================] - 30s 163ms/step - loss: 1.6484
Epoch 4/20
172/172 [==============================] - 30s 164ms/step - loss: 1.5122
Epoch 5/20
172/172 [==============================] - 30s 165ms/step - loss: 1.4299
Epoch 6/20
172/172 [==============================] - 30s 165ms/step - loss: 1.3731
Epoch 7/20
172/172 [==============================] - 30s 165ms/step - loss: 1.3281
Epoch 8/20
172/172 [==============================] - 30s 165ms/step - loss: 1.2893
Epoch 9/20
172/172 [==============================] - 30s 164ms/step - loss: 1.2529
Epoch 10/20
172/172 [==============================] - 30s 164ms/step - loss: 1.2182
Epoch 11/20
172/172 [==============================] - 30s 164ms/step - loss: 1.1827
Epoch 12/20
172/172 [==============================] - 30s 164ms/step - lo

### Making predictions
We'll need to load the model from a checkpoint using a batch size of 1 so we can feed text and make the model predict the next characters.

In [46]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [47]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint(f"./training_checkpoints/ckpt_{checkpoint_num}"))
model.build(tf.TensorShape([1, None]))

### Generating text from input

In [48]:
def generate_text(model, start_string, num_generate=800):
  # Evaluation step (generating text using the learned model)

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [54]:
start_string = input("Type a starting string: ")
length = int(input("Number of characters to generate: "))
print(generate_text(model, start_string, length))

Type a starting string: Yes sir
Number of characters to generate: 20
Yes sir,
Which, brought him
